In [ ]:
import chess
import pyspark.sql.functions as F
import pandas as pd
from IPython.display import display, HTML
import numpy as np
import seaborn as sns
# pyspark.sql.functions.split(df['my_str_col'], '-')

In [ ]:
spark

In [ ]:
raw = spark.read.format("parquet").option("header",True).load("data/chess_dataframe_parquet/*.parquet")

In [ ]:
raw.show()

In [ ]:
sp = F.split(raw.Opening, ':')
df = raw.withColumn("Opening1", sp.getItem(0)).withColumn("Variation", sp.getItem(1))

In [ ]:
df = df.groupby(df.Opening1, df.Variation).count()

In [ ]:
df.take(5)

In [ ]:
x = df.filter(F.col("count")>100).sort(F.col("count").desc()).toPandas()
x.set_index(['Opening1', "Variation"], inplace=True)
x.sort_index(inplace=True)
display(HTML(x.to_html()))

## Total opening count table

In [ ]:
df.groupby(df.Opening1).agg(F.sum("count").alias("count")) \
  .filter(F.col("count")>100).sort(F.col("count").desc()).toPandas()

## Opening with its most popular variation count

In [ ]:
import matplotlib.pyplot as plt
p = df.filter(F.col("count")>2000).filter(F.col("variation").isNotNull()) \
      .sort(F.col("count").desc()) \
      .toPandas()
p.plot(x="Opening1", y="count", kind="bar", rot=90, fontsize=20, figsize=(20,10))
plt.show()

## Most player variation

In [ ]:
import matplotlib.pyplot as plt
p = df.filter(F.col("count")>8000).filter(F.col("variation").isNotNull()) \
      .withColumn("opening_variation", F.concat(F.col("Opening1"), F.col("variation"))).sort(F.col("count").desc()) \
      .toPandas()
p.plot(x="opening_variation", y="count", kind="bar", rot=90, fontsize=20, figsize=(20,10))
plt.show()

## Most played opening

In [ ]:
import matplotlib.pyplot as plt
df2 = raw.withColumn("Opening1", sp.getItem(0))
df2 = df2.groupby(df2.Opening1).count().sort(F.col("count").desc())
p = df2.filter(F.col("count")>10000).toPandas()
p.plot(x="Opening1", y="count", kind="bar", rot=90, fontsize=20, figsize=(20,10))
plt.show()

## Win rate

In [ ]:
res = raw.select(["Result", "Opening"])
sp = F.split(res.Opening, ':')
res = res.withColumn("Opening_temp", sp.getItem(0)).withColumn("Variation", sp.getItem(1))
res = res.drop("Opening")
res = res.withColumnRenamed("Opening_temp", "Opening")

res = res.groupby("Opening", "Result").count()
res = res.withColumn("WonBy", F.when(res.Result == "0-1", F.lit("black")) \
                         .when(res.Result == "1-0", F.lit("white")) \
                         .when(res.Result == "1/2-1/2", F.lit("draw")) \
                         .otherwise(F.lit("aborted"))).drop("Result").sort(F.col("Opening").desc())

In [ ]:
res.show()

In [ ]:
perc = res.groupby("Opening").agg(F.sum("count").alias("total"))

In [ ]:
perc.show()

In [ ]:
from pyspark.sql import Window
w = Window.partitionBy('Opening')
maxCount = res.where(F.col("wonBy") != "draw").withColumn('maxCount', F.max('count').over(w))\
    .where(F.col('count') == F.col('maxCount'))\
    .drop('maxCount')

In [ ]:
draws = res.filter(F.col("wonBy") == "draw").withColumnRenamed("count","draw_count").drop("wonBy")

In [ ]:
draws.show()

In [ ]:
final = perc.join(maxCount, on=["Opening"], how="inner") \
        .join(draws, on=["Opening"], how="inner") \
        .withColumn("win_rate", F.col("count")/F.col("total")) \
        .withColumn("draw_rate", F.col("draw_count")/F.col("total")) \
        .filter(F.col("total")>500).sort(F.col("total").desc()) \
        .drop("count").withColumn("second_rate", 1 - F.col("win_rate") - F.col("draw_rate"))

In [ ]:
a = final.toPandas()
display(HTML(a.to_html()))

## Daily and weekly games count

In [ ]:
grouped = raw.groupby("date", "opening").agg(F.count("*").alias("count")).sort(F.col("date").desc())
grouped.show()

In [ ]:
total_games_daily = grouped.groupby("date").agg(F.sum(F.col("count")).alias("total"))
total_games_daily.show()

In [ ]:
total_games_weekly = grouped.groupby(F.weekofyear(F.to_date(F.col("date"),"yyyy.MM.dd")).alias("week")) \
                            .agg(F.sum(F.col("count")).alias("total"))
total_games_weekly.show()

## Most played 3 game in each day

In [ ]:
w_date = Window.partitionBy('date').orderBy(F.col("count").desc())

p = grouped.select("*", F.row_number().over(w_date).alias('row_number')) \
          .where(F.col('row_number') <= 3).toPandas()
display(HTML(p.to_html()))


## Queen's Gambit daily game count

In [ ]:
queen = grouped.filter(F.col("opening").contains("Queen's Gambit")) \
        .groupby("date").agg(F.sum(F.col("count")).alias("count")) \
        .sort(F.col("date"))
queen = queen.toPandas()
# queen = queen.set_index("date")
queen.plot.line(rot=50)
# display(HTML(queen.to_html()))
sns.lmplot(x="index",y='count',order=2, data=queen.reset_index(),fit_reg=True) 

## Sicilian Defense daily game count

In [ ]:
sicilian = grouped.filter(F.col("opening").contains("Sicilian Defense")) \
        .groupby("date").agg(F.sum(F.col("count")).alias("count")) \
        .sort(F.col("date"))
sicilian = sicilian.toPandas()
# sicilian = sicilian.set_index("date")
sicilian.plot.line(rot=50)
# display(HTML(sicilian.to_html()))
sns.lmplot(x="index",y='count',order=2, data=sicilian.reset_index(),fit_reg=True) 

## Queen's Gambit weekly game count

In [ ]:
temp = grouped.filter(F.col("opening").contains("Queen's Gambit")) \
        .groupby(F.weekofyear(F.to_date(F.col("date"),"yyyy.MM.dd")).alias("week")) \
        .agg(F.sum(F.col("count")).alias("total")) \
        .sort(F.col("week")).toPandas()
temp.drop(temp.tail(1).index,inplace=True)
temp = temp.set_index("week")
temp.plot.line()
sns.lmplot(x="week",y='total',order=2, data=temp.reset_index(),fit_reg=True) 

## Sicilian Defense weekly game count

In [ ]:
temp = grouped.filter(F.col("opening").contains("Sicilian Defense")) \
        .groupby(F.weekofyear(F.to_date(F.col("date"),"yyyy.MM.dd")).alias("week")) \
        .agg(F.sum(F.col("count")).alias("total"))\
        .sort(F.col("week")).toPandas()
temp.drop(temp.tail(1).index,inplace=True)
temp = temp.set_index("week")
temp.plot.line()
sns.lmplot(x="week",y='total',order=2, data=temp.reset_index(),fit_reg=True) 

## English Opening weekly game count after week 30

In [ ]:
temp = grouped.filter(F.col("opening").contains("English Opening")) \
        .groupby(F.weekofyear(F.to_date(F.col("date"),"yyyy.MM.dd")).alias("week")) \
        .agg(F.sum(F.col("count")).alias("total")) \
        .sort(F.col("week")).filter(F.col("week") > 30).toPandas()
temp.drop(temp.tail(1).index,inplace=True)
temp = temp.set_index("week")
temp.plot.line()
sns.lmplot(x="week",y='total',order=2, data=temp.reset_index(),fit_reg=True) 

## Italian Game weekly game count after week 27

In [ ]:
temp = grouped.filter(F.col("opening").contains("Italian Game")) \
        .groupby(F.weekofyear(F.to_date(F.col("date"),"yyyy.MM.dd")).alias("week")) \
        .agg(F.sum(F.col("count")).alias("total")) \
        .sort(F.col("week")).filter(F.col("week") > 27).toPandas()
temp.drop(temp.tail(1).index,inplace=True)
temp = temp.set_index("week")
temp.plot.line()
sns.lmplot(x="week",y='total',order=2, data=temp.reset_index(),fit_reg=True) 

## Weekly game count in all data

In [ ]:
temp = total_games_weekly.sort(F.col("week")).toPandas()
temp.drop(temp.tail(1).index,inplace=True)
temp = temp.set_index("week")
temp.plot.line()
sns.lmplot(x="week",y='total',order=2, data=temp.reset_index(),fit_reg=True) 